In [1]:
import os
import shutil
import random
from pathlib import Path
import yaml
import json

# Один раз загружаем словарь из файла
with open('item_id_map.json', 'r', encoding='utf-8') as f:
    name_to_id = json.load(f)

# Параметры
dataset_dir = Path("dataset")
images_dir = dataset_dir / "images" / "train_full"
labels_dir = dataset_dir / "labels" / "train_full"

train_images_dir = dataset_dir / "images" / "train"
val_images_dir = dataset_dir / "images" / "val"
train_labels_dir = dataset_dir / "labels" / "train"
val_labels_dir = dataset_dir / "labels" / "val"

val_ratio = 0.2  # 20% для валидации

# Создаём папки, если их нет
for d in [train_images_dir, val_images_dir, train_labels_dir, val_labels_dir]:
    d.mkdir(parents=True, exist_ok=True)

# Получаем список всех файлов изображений
all_images = sorted(images_dir.glob("*.png"))  # подстрой под формат, если другой

# Перемешиваем для случайного разбиения
random.shuffle(all_images)

num_val = int(len(all_images) * val_ratio)
val_images = all_images[:num_val]
train_images = all_images[num_val:]

def copy_files(file_list, images_dst, labels_dst):
    for img_path in file_list:
        label_path = labels_dir / (img_path.stem + ".txt")

        shutil.copy(img_path, images_dst / img_path.name)
        if label_path.exists():
            shutil.copy(label_path, labels_dst / label_path.name)
        else:
            print(f"Внимание: нет аннотации для {img_path.name}")

# Копируем файлы
copy_files(train_images, train_images_dir, train_labels_dir)
copy_files(val_images, val_images_dir, val_labels_dir)

print(f"Train: {len(train_images)} изображений")
print(f"Val: {len(val_images)} изображений")

# Генерация data.yaml для YOLOv8
data_yaml = {
    'path': str(dataset_dir.resolve()),  # корневая папка dataset
    'train': str(train_images_dir.relative_to(dataset_dir)),
    'val': str(val_images_dir.relative_to(dataset_dir)),
    'nc': len(name_to_id),  # количество классов, у тебя должно быть определено name_to_id
    'names': [None] * len(name_to_id)
}

# Заполним имена классов в порядке id (предполагается, что name_to_id — dict {name: id})
for name, idx in name_to_id.items():
    data_yaml['names'][idx] = name

yaml_path = dataset_dir / "data.yaml"
with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f, sort_keys=False)

print(f"Сгенерирован файл {yaml_path}")


KeyboardInterrupt: 

In [4]:
import os
import shutil
import random
from pathlib import Path
import json
import yaml
from concurrent.futures import ThreadPoolExecutor  # вместо ProcessPoolExecutor
from tqdm import tqdm

# Один раз загружаем словарь из файла
with open('item_id_map.json', 'r', encoding='utf-8') as f:
    name_to_id = json.load(f)

# Параметры
dataset_dir = Path("dataset")
images_dir = dataset_dir / "images" / "train_full"
labels_dir = dataset_dir / "labels" / "train_full"

train_images_dir = dataset_dir / "images" / "train"
val_images_dir = dataset_dir / "images" / "val"
train_labels_dir = dataset_dir / "labels" / "train"
val_labels_dir = dataset_dir / "labels" / "val"

val_ratio = 0.2  # 20% для валидации

# Создаём папки, если их нет
for d in [train_images_dir, val_images_dir, train_labels_dir, val_labels_dir]:
    d.mkdir(parents=True, exist_ok=True)

# Получаем список всех файлов изображений
all_images = sorted(images_dir.glob("*.png"))
random.shuffle(all_images)

num_val = int(len(all_images) * val_ratio)
val_images = all_images[:num_val]
train_images = all_images[num_val:]



def copy_pair(img_path_str, images_dst_str, labels_dst_str):
    try:
        img_path = Path(img_path_str)
        images_dst = Path(images_dst_str)
        labels_dst = Path(labels_dst_str)

        label_path = labels_dir / (img_path.stem + ".txt")
        shutil.copy(img_path, images_dst / img_path.name)
        if label_path.exists():
            shutil.copy(label_path, labels_dst / label_path.name)
            return f"OK: {img_path.name}"
        else:
            return f"Нет аннотации: {img_path.name}"
    except Exception as e:
        return f"Ошибка: {img_path.name} — {e}"

def copy_files_parallel(file_list, images_dst, labels_dst, desc="Копирование"):
    images_dst_str = str(images_dst)
    labels_dst_str = str(labels_dst)
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(copy_pair, str(img), images_dst_str, labels_dst_str)
            for img in file_list
        ]
        for f in tqdm(futures, desc=desc):
            msg = f.result()
            if not msg.startswith("OK"):
                print(msg)

# Копируем с прогрессбаром и многопроцессорностью
copy_files_parallel(train_images, train_images_dir, train_labels_dir, desc="Train")
copy_files_parallel(val_images, val_images_dir, val_labels_dir, desc="Val")

print(f"Train: {len(train_images)} изображений")
print(f"Val: {len(val_images)} изображений")

# Генерация data.yaml
data_yaml = {
    'path': str(dataset_dir.resolve()),
    'train': str(train_images_dir.relative_to(dataset_dir)),
    'val': str(val_images_dir.relative_to(dataset_dir)),
    'nc': len(name_to_id),
    'names': [None] * len(name_to_id)
}
for name, idx in name_to_id.items():
    data_yaml['names'][idx] = name

yaml_path = dataset_dir / "data.yaml"
with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f, sort_keys=False)

print(f"Сгенерирован файл {yaml_path}")


Val: 100%|██████████| 2000/2000 [01:43<00:00, 19.40it/s]

Train: 8000 изображений
Val: 2000 изображений
Сгенерирован файл dataset\data.yaml
